In [15]:
import lxml.html
import urllib.request as ur
import requests
import time
import re
import os

In [3]:
link = "http://www.irk.kp.ru/online/news/2644091/"
resp = ur.urlopen(link)
page = resp.read().decode("utf-8")
resp.close()

In [4]:
tree = lxml.html.fromstring(page)

In [7]:
#xpath - 
# точка - значит корень дерева, а два слеша - любой уровень вложенности дерева, а не от корня
title = tree.findtext(".//title")
print(title)

Проезд по улице Карла Либкнехта ограничат на одну ночь в Иркутске


In [11]:
#'pubdate'
date = tree.xpath('.//script[@type="text/javascript"]')[0].get("pubdate")
print(date)

None


In [12]:
#"author"
author = tree.xpath('.//a[@title]/text()') #текст изнутри тега
print(author)

['Политика', 'Общество', 'Экономика', 'В МИРЕ', 'Звезды', 'Здоровье', 'АФИША', 'Наука', 'Авто', 'Недвижимость', 'МАГАЗИН', 'Работа!', 'Андрей СИНЬКОВ', 'Напечатать', 'VKontakte', 'Google Pluse', 'Одноклассники', 'Мой мир', 'Facebook', 'Политика', 'Экономика', 'Общество', 'Колумнисты', 'Происшествия', 'Звезды', 'Здоровье', 'Спецпроекты', 'Спорт', 'Туризм', 'Есть работа!', 'Пресс-центр', 'Наука', 'Недвижимость', 'Подписка', 'Телевизор', 'История современности', 'ИНТЕРНЕТ-МАГАЗИН', 'Коллекции', 'Фото', 'Видео', 'Алла Пугачева', 'Афиша', 'В мире', 'Турецкие цитрусовые', 'ВИЧ: Знать, чтобы жить', 'Всё о наших пенсиях', 'Гагарин', 'Год экологии', 'Грипп', 'Дача', 'Дом. Семья', 'Жанна Фриске', 'Здоровое долголетие', 'Земля леопарда', 'Книжная полка', 'Комфортный шоппинг', 'Курс рубля', "Money' s life", 'Мама и малыш', 'Минздрав предупреждает', 'Наши права и льготы', 'Образование', 'Перевал Дятлова', 'Поддержка семьи', 'Приятного аппетита', 'Энергетика в деталях', 'Радио «Комсомольская Правда»

In [14]:
paragraf = tree.xpath('.//div[@itemprop="articleBody"]/p/text()')
print(paragraf)

['Внимание, иркутские автомобилисты! С 22.00 часов 2 февраля до 06.00 3 февраля частично ограничат проезд по улице Карла ', ' возле пересечения с улицей Дзержинского.', 'Как сообщили в пресс-службе администрации ', ', ограничение для движения транспорта связано с тем, что рабочие будут проводить аварийные ремонтные работы на сетях водопровода', 'Кроме этого, с 1 по 15 февраля в Иркутске ', ' по улице Набережная ', ' в районе КПП терминально-складского комплекса. Там, рабочие также проведут аварийные работы на водопроводных сетях.']


### Задание

скачать КП
тексты с июля 2015 по декабрь 2016
сами тексты + метаданные в таблице csv табуляцией
с названием статьи, автор, количество слов, путь к файлу
сами тексты должны быть распределены по папкам по году и по месяцу в году
с помощью lxml
потом разметить морфологически


In [16]:
import lxml.html
import urllib.request as ur
import requests
import time
import re
import os

In [25]:
#startlink - первая ссылка, с которой мы начнем наш обход ресурса
startlink = "http://www.kp.ru/online/news/2644470/"
resp = ur.urlopen(startlink)
page = resp.read().decode("utf-8")
resp.close()

In [18]:
tree = lxml.html.fromstring(page)

In [22]:
# надо создать папки с годами - 2015 и 2016, и в каждой по 12 папок с месяцами

for year in range(2015,2017):
    for month in range(1,13):
        if month < 10:
            month = "0" + str(month)
        path = "/home/mi_air/Downloads/Komsomol_Truth/"+str(year)+"/"+str(month)
        os.makedirs(path, mode=0o777, exist_ok=False)

In [27]:
# файлы будем называть айдишниками текстов, так как они уникальные
# обход начнем с указанного номера 
num = int(startlink.split("/")[5])

In [36]:
#напишем функцию, которая будет проверять, находится ли по указанной ссылке текст, в нужном интервале дат:
# с июля 2015 по декабрь 2016 (июль 2015 включаем, декабрь 2016 не включаем)
# заодно она будет на м мгод с месяцем возвравщать, чтобы удобно было сохранять текст в нужную папку
def TextDate(link):
    bl = False
    r = requests.get(link)
    date=re.split("T", re.split("'pubdate':'", r.text)[1])[0]
    year = int(re.split("-", date)[0])
    month = int(re.split("-", date)[1])
    if year==2016:
        if month<12:
            bl=True
    if year==2015:
        if month>5:
            bl=True
    return [bl,year,month]

In [37]:
#проверка этой функции
print(TextDate("http://www.kp.ru/online/news/2644468/"))
print(TextDate("http://www.kp.ru/online/news/1844468/"))
print(TextDate("http://www.kp.ru/online/news/2144468/"))

[False, 2017, 2]
[False, 2014, 9]
[True, 2015, 8]


In [ ]:
#заодно мы выяснили, что в районе 1,8 млн статей нужные нам статьи уже заканчиваются, 
#поэтому не будет заставлять программу перебирать номера статей так долго и ограничим их этим числом
while num<1844468:
    link = "http://www.kp.ru/online/news/"+str(num)+"/"
    if DateText(link)[0]==True:
        try:
            #[n_time, n_date, n_rubr, n_id, n_author, n_head, n_region, n_text]
            params = getArticleTextKP(link)
            corpus.write("<author>"+params[4]+"</author>"+"\n")
            corpus.write("<textname>"+params[5]+"<textname>"+"\n")
            corpus.write("<textregion>"+params[6]+"<textregion>"+"\n")
            corpus.write("<textrubric>"+params[2]+"</textrubric>"+"\n")
            corpus.write("<date>"+params[1]+"</date>"+"\n")
            corpus.write("<time>"+params[0]+"</time>"+"\n")
            corpus.write("<source>"+link+"</source>"+"\n")
            corpus.write("<textid>" + params[3]+"</textid>" +"\n")
            corpus.write("<text>"+"\n")
            corpus.write(params[7]+"\n")
            corpus.write("</text>"+"\n")
            print(link)
            hreffile.write(link+"\n")
        except:
            pass
    num-=1

In [23]:
# Функия принимает адрес статьи на KP u и возвращает текст статьи и метаинформацию по ней.
def getArticleTextKP(adr):
    r = requests.get(adr) # Получаем текст статьи и выкусываем все нужное
    
    n_time=re.split("T", re.split("',", re.split("'pubdate':'", r.text)[1])[0])[1]
    n_date=re.split("T", re.split("'pubdate':'", r.text)[1])[0]
    n_rubr=re.split("',", re.split("'sections':'", r.text)[1])[0]
    n_id = re.split("',", re.split("'post_id':'", r.text)[1])[0]
    n_region = re.split("',", re.split("'ci_region':'", r.text)[1])[0]
    n_head=re.split("',",re.split("'title': '", r.text)[1])[0]
    n_author=re.split("',",re.split("authors': '", r.text)[1])[0]
    text=re.split("</div></article>",re.split('itemprop="articleBody" id="hypercontext">', r.text)[1])[0] 

    #чистим текст от тегов
    beaux_text=BeautifulSoup(text, "lxml")
    n_text=beaux_text.get_text() 
    time.sleep(random.uniform(1, 3))

    return [n_time, n_date, n_rubr, n_id, n_author, n_head, n_region, n_text]


mystem ожидает на вход utf-8
/home/mi_air/mystem
для бинарника на убунту нужно после скачивания сделать chmod 777 путь

ctrl+c - заверщение работы программы

mystem input output

без настроек - только лемматизация, убираются межсловные интервалы, знаки препинания, переносы строк...

надо юзать опции! https://tech.yandex.ru/mystem/doc/index-docpage/ 

In [1]:
! /home/mi_air/mystem 1.txt 2.txt

In [2]:
! /home/mi_air/mystem -ni -d 1.txt 1_out.txt

In [3]:
! /home/mi_air/mystem -ni -d --weight 1.txt 1_weighted.txt

In [4]:
import os
os.system('/home/mi_air/mystem -ni -d 1.txt 1_os.txt')

0

Новое условие - по 5 млн слв в год, минимум 7,5 млн
В метатаблице должны быть тексты по всем месяцам и годам

